### Python scraping

```
personal notebook about webscraping using Selenium, beautifiul soup , and normal http lib request.

A website has 3 main components , html for contents, javascript for interaction, css for style.

Why scraping: Automate data gathering

css selectors: id, tags and classes.
tags is for example <p> , which css can find using p {}
<p id="mijnid"> can be found using #id . 
<p class="mijnclass"> can be found using .mijnclass , and is used for many tags with the same class.

To debug: open Chrome , and go to inspect components on the website
In console use $$(".name") to find css tags to verify/debug your finding.

In console open your item to see all attributes. 
use :
[...$$(".yourname")].forEach( (e) =>: { } ];

it shows your attribute.


to scrape install selnium:
python -m pip install selenium
Also a webdriver (chrome driver) can be used to test websites. In the end we will just use a aorchestration tool which call our scraping tool periodically. But before that moment we will use chromedriver to research the website using code
```

In [72]:
from selenium import webdriver
browser = webdriver.Chrome("/Users/michelnossin/Downloads/chromedriver")
#browser.get("https://github.com/michelnossin/python-scraping/commits/master")
browser.get("https://github.com/michelnossin/commander/commits/master")
#browser.execute_script("alert('hello michel')")
#input.send_keys("some input text for input box")
#input.clear()

In [75]:
import copy


def retrieve_lists_commitinfo ():
  
   dates = browser.find_elements_by_css_selector(".commit-meta")
   titles = browser.find_elements_by_css_selector(".commit-title")
   authors = browser.find_elements_by_css_selector(".commit-author")
   alist = [a.text for a in authors]
   tlist = [t.text for t in titles]
   dlist = [d.text for d in dates]
   dat = zip(dlist,alist,tlist)
   dat = [list(a) for a in dat]
    #return dat
   deeper_list=[]

   try:
      older_button = browser.find_element_by_css_selector(".pagination a")
      older_button.click()
      deeper_list = retrieve_lists_commitinfo()
   except:
      return dat
    
   return dat + deeper_list

In [76]:
dat = retrieve_lists_commitinfo()
[x for x in dat]

[['michelnossin committed on Feb 25',
  'michelnossin',
  'change readme to deploy on aws production'],
 ['michelnossin committed on Feb 22',
  'michelnossin',
  'fixed some bug in monitoring events within dialog, part2'],
 ['michelnossin committed on Feb 22',
  'michelnossin',
  'fixed some bug in monitoring events within dialog'],
 ['michelnossin committed on Feb 22',
  'michelnossin',
  'first events now in context dialog received'],
 ['michelnossin committed on Feb 19',
  'michelnossin',
  'Dialog handles topic changes using combobox'],
 ['michelnossin committed on Feb 15',
  'michelnossin',
  'parsed topics and showed within contextdialog, to beautify later'],
 ['michelnossin committed on Feb 14',
  'michelnossin',
  'Can now receive topics kafka'],
 ['michelnossin committed on Feb 14',
  'michelnossin',
  'fixed events kafka within context dialog'],
 ['Michel Nossin committed on Feb 14',
  'Michel Nossin',
  'Events are hown in contextmenu'],
 ['michelnossin committed on Feb 12',

To Browse to multiple pages, you can find the button in the same matter,
and use .click() to push next. If a button does no have a css selector,
but it sits in a another item like a span tag, you can use e.g. $$(".span a") in chrome to find the link item in a span item.